<font color='green'>

## 16. Pivot Tablolar

### Pivot table bir tablodaki verileri yatay olarak göstermemizi saglar yani satirlarda bulunan bilgileri istedigimiz alanlara göre sutunlara çevirir.  Özellikle rapor hazirlarken verileri daha anlasilir ve düzenli hale getirmemizi saglar. Örnegin bir siparis tablomuz var. Bir müsterinin verdigi her bir siparis bir satirda gösteriliyor. Her müsteri için son 6 aylik siparis bilgilerini görmek istiyoruz. Birinci yöntem her hareketi alt alta yazmak olabilir ama daha anlasilir ve düzenli bir rapor için her müsteri bir satirda yer almali ve bu satirda son alti aya ait tüm siparis tutarlarinin toplami gösterilmelidir. Bu durumda pivot table kullanmak imdadimiza yetisir.

### Excel çalışma sayfalarındaki, SQL Server gibi programlardaki verilerizi hızlı bir şekilde verilerinizi özetlemek, çözümlemek, keşfetmek ve sunmak için mükemmel bir yoldur.

![](https://exceljet.net/sites/default/files/images/term/pivot%20table.png)

### Amaç veri setini amaca uygun hale getirmek için kullanılan yapılardır. Ancak groupby() ile karıştırılmamalıdır.

In [1]:
import pandas as pd
import seaborn as sns

### Farklı olarak burada seaorn kütüphanesinden de birkaç fonksiyon kullanacağımız için onu da import ettik.

In [4]:
titanic=sns.load_dataset('titanic')

###  Hali hazırda bulunan 'titanic' veri setini çektik ve bunu bir değişkene atadık.

### seaborn kütüphanesindeki başka veri setlerini görmek için [buraya tıklayabilirsiniz.](https://github.com/mwaskom/seaborn-data)

### Veri setimiz çok uzun olduğu için ilk 5 verisini görelim. Veri setimizi ve değişkenlerimizi tanıyalım.

In [5]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
titanic.groupby("sex")["survived"].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

### Cinsiyete göre gruplama işlemi yaptık. Sürekli değişken olarak 'survived' değişkenini seçtik ve ortalamasını al dedik. Veri seti hayatta kalmayı '1' ölmeyi '0' almıştır. '1' ve '0' lardan oluşan bu veri setinin ortalamasını aldık.

### Şimdi ise bu bilgiyi bir dataframe haline getirelim.

In [7]:
titanic.groupby("sex")[["survived"]].mean()

,survived
sex,
female,0.742038
male,0.188908


### Basit bir pivot işlemi yaptık.

### Bu örnek çok ilkelce bir örnek oldu. Bir gruplama daha ekleyerek işlemi ilerletelim.

### Bir grup daha ekleyerek iki boyutta bir pivot tablo oluşturalım :

In [9]:
titanic.groupby(["sex","class"])[["survived"]].aggregate("mean")
titanic.groupby(["sex","class"])[["survived"]].aggregate("mean").unstack()

survived                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

### groupby() aracılığıyla bir pivot işlemi yaptık. Yani veri setini x ekseninde cinsiyet kategorik değişkeni ve y eksenininde class kategorik değişkeninin sınıflarına göre bölüp kesişimlerinden bir veri elde ettik.

### Hem cinsiyete göre hem de class'a göre grupla. Sonrasında aggregate ile bu kesişimden oluşan verinin ortalamasını buldurduk. Aggregate uygulamaya alışırsak başka bir bilgi istediğimiz zaman da yapımız kurulu olacak ve sadece istediğimiz işlemi yazmamız yeterli olacaktır. .unstack() kısmına gelecek olursak kullanımının amacı hiyerarşik index yapısını çözmek ve tablonun okunmasını kolaylaştırmaktır.

### Bu noktaya kadar pivot tale oluşturmanın zor yolunu gördük. Şimdi ise daha kolay bir yöntemle pivot table oluşturalım. Bunun için .pivot_table() fonksiyonunu kullanacağız.

In [10]:
titanic.pivot_table("survived", index="sex",columns = "class")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


### bize diyor ki senin için önemli olan,odaklandığın değişken nedir? "survived" diyoruz. index'imiz cinsiyet, başka hangi değişkene odaklanmak istersin? "column" index olarak yazdığımız yeri x ekseni olarak alıyor.

### Çok boyutlu bir pivot table oluşturalım.

In [11]:
titanic.age.head()

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: age, dtype: float64

### Bu sürekli değişkeni bir kategorik değişkene çevirelim. Bu kategorik değişkenin sınıflarını da pivot tabloya boyut olarak ekleyelim :

In [12]:
age=pd.cut(titanic["age"],[0, 18, 90])
age.head(10)

0    (18.0, 90.0]
1    (18.0, 90.0]
2    (18.0, 90.0]
3    (18.0, 90.0]
4    (18.0, 90.0]
5             NaN
6    (18.0, 90.0]
7     (0.0, 18.0]
8    (18.0, 90.0]
9     (0.0, 18.0]
Name: age, dtype: category
Categories (2, interval[int64]): [(0, 18] < (18, 90]]

### sürekli değişkeni kategorik değişkene çevirdik ve bu kategorik değişkene 3 tane sınıf oluşturduk. Peki bu sınıfları bir boyut olarak pivot table'a nasıl ekleyebiliriz?

In [13]:
titanic.pivot_table("survived",["sex",age], "class")

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 90]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 90]  0.375000  0.071429  0.133663

### Kendi oluşturmuş olduğumuz bir kategorik değişkeni bir boyut olacak şekilde basitçe tabloya eklemiş olduk. Kullanmış olduğumuz kategorik değişkeni böyle direkt yazmamız mevcut yapısını koruyarak almış olduğu için istediğimiz tabloyu elde ederiz. age'yi dışarıdan aldığımız için böyle yazdık.

### [17. Dış Kaynaklı Veri Okuma notlarına geçmek için tıklayınız.](https://github.com/semihstp/Veri-Bilimi-Not-Defterim/blob/master/Pandas%20K%C3%BCt%C3%BCphanesi/17.%20D%C4%B1%C5%9F%20Kaynakl%C4%B1%20Veri%20Okuma.ipynb)